# Разработка модели машинного обучения

1. **Разработка модели машинного обучения**
   1. Обучающая выборка данных разделена на обучающую и тестовую выборку.
   2. Испытано как минимум три алгоритма разной сущности (например, `RandomForest` и `XGBoost` – разные сущности, а `XGBoost` и `GradientBoosting` – нет). Для оценки точности следует использовать тестовую выборку.
   3. Испытанные алгоритмы сравнены по точности, а также по скорости работы.
   4. Лучший алгоритм реализован в виде пайплайна, принимающего на вход пути к файлам, а на выходе возвращает обученную модель, а также метрики модели на валидационной выборке в виде словаря (необходимые метрики: средний коэффициент детерминации **R²** и средний **RMSE**).

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from time import time
import os

In [9]:
# 1. Загрузка данных
def load_data(base_path, data_files, target_files, nrows=10000):
    """
    Загружает данные из CSV-файлов и объединяет их.
    :param base_path: Путь к папке с файлами.
    :param data_files: Список файлов данных.
    :param target_files: Список файлов целевых значений.
    :param nrows: Количество строк для загрузки из каждого файла.
    :return: Объединенные данные и целевые значения.
    """
    # Загрузка данных
    data_dfs = []
    for file in data_files:
        file_path = os.path.join(base_path, file)
        df = pd.read_csv(file_path, nrows=nrows)  # Загружаем первые nrows строк
        data_dfs.append(df)

    # Объединение данных
    combined_data = pd.concat(data_dfs, ignore_index=True)

    # Загрузка целевых значений
    target_dfs = []
    for file in target_files:
        file_path = os.path.join(base_path, file)
        df = pd.read_csv(file_path, nrows=nrows)  # Загружаем первые nrows строк
        target_dfs.append(df)

    # Объединение целевых значений
    combined_target = pd.concat(target_dfs, ignore_index=True)

    return combined_data, combined_target


# 2. Разделение данных на обучающую и тестовую выборки
def split_data(data, target, test_size=0.2, random_state=42):
    """
    Разделяет данные на обучающую и тестовую выборки.
    :param data: Признаки (DataFrame).
    :param target: Целевая переменная (DataFrame или Series).
    :param test_size: Доля тестовой выборки.
    :param random_state: Случайное состояние для воспроизводимости.
    :return: X_train, X_test, y_train, y_test
    """
    X_train, X_test, y_train, y_test = train_test_split(
        data, target, test_size=test_size, random_state=random_state
    )
    return X_train, X_test, y_train, y_test


# 3. Оценка модели
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    Обучает модель и оценивает её производительность.
    :param model: Модель машинного обучения.
    :param X_train, X_test, y_train, y_test: Данные для обучения и тестирования.
    :return: Словарь с метриками и временем выполнения.
    """
    start_time = time()
    model.fit(X_train, y_train)
    training_time = time() - start_time

    start_time = time()
    y_pred = model.predict(X_test)
    prediction_time = time() - start_time

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Расчет процента улучшения по сравнению с базовым уровнем (средним значением y_test)
    baseline_rmse = np.sqrt(mean_squared_error(y_test, np.full_like(y_test, y_test.mean())))
    improvement_percentage = ((baseline_rmse - rmse) / baseline_rmse) * 100

    return {
        "R2": r2,
        "RMSE": rmse,
        "Improvement %": improvement_percentage,
        "Training Time": training_time,
        "Prediction Time": prediction_time
    }


# 4. Сравнение алгоритмов
def compare_models(X_train, X_test, y_train, y_test):
    """
    Сравнивает несколько алгоритмов машинного обучения.
    :param X_train, X_test, y_train, y_test: Данные для обучения и тестирования.
    :return: Список результатов для каждого алгоритма.
    """
    models = {
        "RandomForest": RandomForestRegressor(random_state=42),
        "XGBoost": XGBRegressor(random_state=42),
        "LinearRegression": LinearRegression()
    }

    results = []
    for name, model in models.items():
        print(f"Обучение модели: {name}")
        metrics = evaluate_model(model, X_train, X_test, y_train, y_test)
        results.append({
            "Model": name,
            "R2": metrics["R2"],
            "RMSE": metrics["RMSE"],
            "Improvement %": metrics["Improvement %"],
            "Training Time": metrics["Training Time"],
            "Prediction Time": metrics["Prediction Time"]
        })

    return results


# 5. Реализация пайплайна
def pipeline(base_path, data_files, target_files, nrows=10000):
    """
    Пайплайн для обучения модели и оценки её производительности.
    :param base_path: Путь к папке с файлами.
    :param data_files: Список файлов данных.
    :param target_files: Список файлов целевых значений.
    :param nrows: Количество строк для загрузки из каждого файла.
    :return: Обученная модель, метрики.
    """
    # Загрузка данных
    data, target = load_data(base_path, data_files, target_files, nrows)

    # Разделение данных
    X_train, X_test, y_train, y_test = split_data(data, target)

    # Сравнение моделей
    results = compare_models(X_train, X_test, y_train, y_test)

    # Вывод результатов
    print("\nСравнение моделей:")
    for result in results:
        print(result)

    # Выбор лучшей модели по метрике R2
    best_model_result = max(results, key=lambda x: x["R2"])
    best_model_name = best_model_result["Model"]

    # Обучение лучшей модели
    if best_model_name == "RandomForest":
        best_model = RandomForestRegressor(random_state=42)
    elif best_model_name == "XGBoost":
        best_model = XGBRegressor(random_state=42)
    elif best_model_name == "LinearRegression":
        best_model = LinearRegression()

    best_model.fit(X_train, y_train)

    # Оценка лучшей модели
    y_pred = best_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Расчет процента улучшения для лучшей модели
    baseline_rmse = np.sqrt(mean_squared_error(y_test, np.full_like(y_test, y_test.mean())))
    improvement_percentage = ((baseline_rmse - rmse) / baseline_rmse) * 100

    metrics = {
        "R2": r2,
        "RMSE": rmse,
        "Improvement %": improvement_percentage
    }

    return best_model, metrics


# Пример использования пайплайна
if __name__ == "__main__":
    # Пути к файлам
    base_path = "/home/c3/Загрузки/BIG DATA"
    data_files = ["df_0.csv", "df_1.csv", "df_2.csv", "df_3.csv"]
    target_files = ["target_0.csv", "target_1.csv", "target_2.csv", "target_3.csv"]

    # Запуск пайплайна
    model, metrics = pipeline(base_path, data_files, target_files, nrows=10000)
    print("\nЛучшая модель:", model)
    print("Метрики на тестовой выборке:", metrics)

Обучение модели: RandomForest
Обучение модели: XGBoost
Обучение модели: LinearRegression

Сравнение моделей:
{'Model': 'RandomForest', 'R2': 0.782953276921003, 'RMSE': np.float64(572.7670925340476), 'Improvement %': np.float64(52.91458756797969), 'Training Time': 77.36223673820496, 'Prediction Time': 0.1855452060699463}
{'Model': 'XGBoost', 'R2': 0.8209034204483032, 'RMSE': np.float64(524.5445643603601), 'Improvement %': np.float64(56.87881256827999), 'Training Time': 0.5628821849822998, 'Prediction Time': 0.007448911666870117}
{'Model': 'LinearRegression', 'R2': 0.3818804483825242, 'RMSE': np.float64(957.2938461848696), 'Improvement %': np.float64(21.303831603122013), 'Training Time': 0.019370555877685547, 'Prediction Time': 0.0009703636169433594}

Лучшая модель: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categori